In [591]:
import pandas as pd
import glob
import os
import numpy as np

path = r'../data/curated/merged_dataset/' # use your path
all_files = glob.glob(os.path.join(path , "*.csv"))

li = []

for filename in sorted(all_files):
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

merged_df = pd.concat(li, axis=0, ignore_index=True)
# log candidates: income_per_person, population_density, crime_cases, weekly_rent
merged_df['income_per_person'] = np.log(merged_df['income_per_person'])
# merged_df['population_density'] = np.log(merged_df['population_density'])
merged_df['crime_cases'] = np.log(merged_df['crime_cases'])
merged_df['weekly_rent'] = np.log(merged_df['weekly_rent'])
merged_df.drop(['address', 'latitude', 'longitude', 'postcode', 'sa2_2016'], axis=1, inplace=True)


In [605]:
# df = merged_df.fillna(300)
# Example of solo SA2 instance
df = merged_df.dropna()
df.loc[df['sa2_2021']==206061516]

,year,sa2_2021,residence_type,nbed,nbath,ncar,min_distance_to_cbd,min_distance_to_park,min_distance_to_prim,min_distance_to_second,min_distance_to_train,min_distance_to_hosp,min_distance_to_poli,min_distance_to_shop,weekly_rent,gdp(USD Millioins),saving_rate(% of GDP),income_per_person,population_density,crime_cases
132662,2022,206061516,Apartment,3.0,1.0,1,6.72465,0.96823,1.43268,114.77016,0.60096,140.56888,74.35608,13.6492,6.39693,3305754,12.839,11.511549,7698.0,6.660575


In [593]:
#df.iloc[132647:]
# drop the SA2 only has one instance
# lost 3 instances
df = df[df.duplicated(subset=["sa2_2021"], keep=False)]
df

,year,sa2_2021,residence_type,nbed,nbath,ncar,min_distance_to_cbd,min_distance_to_park,min_distance_to_prim,min_distance_to_second,min_distance_to_train,min_distance_to_hosp,min_distance_to_poli,min_distance_to_shop,weekly_rent,gdp(USD Millioins),saving_rate(% of GDP),income_per_person,population_density,crime_cases
0,2013,204011057,House,2.0,1.0,0,227.97163,23.16035,7.35747,16.96507,35.56825,21.35025,22.04660,9.35209,5.703782,1536454,6.861393,10.588692,2.172408,4.454347
1,2013,205051101,House,2.0,1.0,0,223.66084,5.71742,6.50536,6.76794,7.54355,7.42972,6.28177,9.35209,5.370638,1536454,6.861393,10.762630,5.425503,3.583519
2,2013,204011057,House,2.0,1.0,0,243.25680,5.11222,0.20027,36.72106,50.85341,36.63541,0.08478,9.35209,5.164786,1536454,6.861393,10.588692,2.172408,4.454347
3,2013,202011022,House,4.0,2.0,0,140.35827,78.32509,10.66523,11.91899,11.26906,177.44731,84.47341,9.35209,5.857933,1536454,6.861393,10.681809,473.765281,7.160846
4,2013,208041195,Apartment,1.0,1.0,0,13.86135,0.93250,1.32931,3.49174,2.20800,177.44731,84.47341,3.96501,5.616771,1536454,6.861393,11.363304,2834.210526,7.561642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172030,2022,205021086,House,3.0,1.0,1,293.28053,0.56012,1.21809,114.77016,90.08591,140.56888,74.35608,13.64920,5.579730,3305754,12.839000,10.903481,402.000000,5.638355
172031,2022,217041479,House,3.0,2.0,2,258.29111,3.49087,5.08707,3.60570,8.37185,2.60312,74.35608,13.64920,6.214608,3305754,12.839000,11.015813,689.000000,8.022569
172032,2022,208021177,House,2.0,2.0,1,9.47077,2.45011,1.33931,1.62322,3.63291,140.56888,74.35608,1.97636,6.620073,3305754,12.839000,11.500412,3656.000000,6.632002
172033,2022,206041506,Apartment,1.0,1.0,1,1.84933,0.65199,1.10438,1.27940,1.87840,140.56888,74.35608,13.64920,6.013715,3305754,12.839000,11.174728,5791.000000,7.488853


In [594]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [595]:
TARGET_COLS = ['weekly_rent']
COLS = list(df.columns)
df = df[COLS]
df = pd.get_dummies(df, columns=['residence_type', 'sa2_2021', 'nbed', 'nbath', 'ncar'])

train, test = train_test_split(df, train_size=0.8, random_state=0)

X_train, y_train = train.drop(TARGET_COLS, axis=1), train[TARGET_COLS]
X_test, y_test = test.drop(TARGET_COLS, axis=1), test[TARGET_COLS]

print(f'{len(X_train)} training instances, {len(X_test)} test instances')

137612 training instances, 34403 test instances


In [596]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, Normalization

In [597]:
# Setup a normalization layer and adapt it to the training set so that it knows
# what mean and sd to use when normalising
norm_layer = Normalization()
norm_layer.adapt(X_train)

In [598]:
model = keras.Sequential(
    [   
        norm_layer,                   # our normalisation layer recieves the input
        Dense(600, activation='relu'),  # the hidden layer gets the normalised result
        Dense(300, activation='relu'),  # (in case you want to try an extra hidden layer)
        Dense(60, activation='relu'),
        # Dense(1, activation='relu')   # and the output layer has a single node which will estimate total_amount
        # Dense(2, activation='softmax')
        # Dense(1, activation='sigmoid')
        Dense(1, activation='relu')
    ]
)

In [599]:
model.compile(
    optimizer='adam',  # Adam optimises using gradient descent, is generally fast and a good choice in many cases
    loss='MSE' # 'sparse_categorical_crossentropy'#'MSE'  # Mean Squared Error makes sense for this problem, 
                # though we could use Mean Absolute Error, or many other choices.
                # Classification outputs would use a different loss (eg. BinaryCrossentropy)
)
# base_learning_rate = 0.0001
# model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate), metrics=['acc'])

In [600]:
history = model.fit(
    x=X_train,
    y=y_train,
    batch_size=16,
    validation_split=0.25,
    epochs=10
)

Epoch 1/10
6451/6451 [==============================] - 12s 2ms/step - loss: 0.1989 - val_loss: 0.2169
Epoch 2/10
6451/6451 [==============================] - 12s 2ms/step - loss: 0.0953 - val_loss: 0.1126
Epoch 3/10
6451/6451 [==============================] - 13s 2ms/step - loss: 0.0739 - val_loss: 0.4786
Epoch 4/10
6451/6451 [==============================] - 12s 2ms/step - loss: 0.0653 - val_loss: 0.1987
Epoch 5/10
6451/6451 [==============================] - 12s 2ms/step - loss: 0.0592 - val_loss: 0.2917
Epoch 6/10
6451/6451 [==============================] - 12s 2ms/step - loss: 0.0582 - val_loss: 0.1396
Epoch 7/10
6451/6451 [==============================] - 12s 2ms/step - loss: 0.0550 - val_loss: 0.0880
Epoch 8/10
6451/6451 [==============================] - 13s 2ms/step - loss: 0.0547 - val_loss: 0.0569
Epoch 9/10
6451/6451 [==============================] - 12s 2ms/step - loss: 0.0518 - val_loss: 0.0568
Epoch 10/10
6451/6451 [==============================] - 12s 2ms/step - l

In [601]:
comparison = y_test.iloc[:100].copy()
comparison.loc[:, 'prediction'] = model.predict(X_test.head(100))
comparison

4/4 [==============================] - 0s 2ms/step


,weekly_rent,prediction
89826,5.703782,6.101227
34773,5.752573,5.808753
147964,5.652489,6.123927
121458,6.016157,6.072337
7741,5.991465,5.894453
...,...,...
155773,6.131226,6.509422
44590,5.480639,5.547657
169783,5.991465,5.978645
26312,5.752573,5.800710


In [602]:
model.evaluate(
    x=X_test.iloc[:100],
    y=y_test.iloc[:100],
    batch_size=16,
)

7/7 [==============================] - 0s 1ms/step - loss: 0.0368


0.03676052391529083

In [603]:

predictions = model.predict(X_test.iloc[:])
errors = np.array(predictions - y_test.iloc[:])
squared_errors = errors**2
mean_squared_error = squared_errors.mean()

print(f'MSE: {mean_squared_error}')

1076/1076 [==============================] - 1s 759us/step
MSE: 0.05256338073420245


In [604]:
tot_sum_squares = (np.array(y_test - y_test.mean())**2).sum()
r2 = 1 - (squared_errors.sum() / tot_sum_squares)
print(f'Model R^2: {r2:.4f}')

Model R^2: 0.6450
